In [ ]:
from grid.refine import *

df = observed_rates()

keys = {}
for i, r in df.iterrows():
    ks = {}
    for color in ['black', 'white']:
        name = r[f'{color}_name']
        key = f'{color}_key'
        if name.startswith('mohex'):
            ks[key] = f'{r.boardsize}-{name}'
        elif name == 'latest':
            ks[key] = f'{r.boardsize}B{r.width}W{r.depth}D+S'
        else:
            idx = name.split('.')[1] 
            ks[key] = f'{r.boardsize}B{r.width}W{r.depth}D{idx}S'
    keys[i] = ks
df = pd.concat([df, pd.DataFrame.from_dict(keys, orient='index')], 1)
df = df[['black_key', 'white_key', 'black_wins', 'white_wins']]

for b in [3, 5, 7, 9]:
    aux = database.pandas(f'mohex-{b}').reset_index()
    aux['black_key'] = f'{b}-' + aux.black_name
    aux['white_key'] = f'{b}-' + aux.white_name
    df = pd.concat([df, aux.reindex(columns=df.columns)])

black_wins = pd.pivot_table(df, 'black_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)
white_wins = pd.pivot_table(df, 'white_wins', 'black_key', 'white_key', aggfunc='sum').fillna(0)

wins = black_wins + white_wins.T
games = black_wins + white_wins + black_wins.T + white_wins.T

targets = set(df.black_key.values) | set(df.white_key.values)
targets = [t for t in targets if 'mohex' not in t]

In [ ]:
with aljpy.parallel(solve) as p:
    results = p.wait({t: p(t, games, wins) for t in targets[:5]})

In [ ]:
pd.DataFrame.from_dict(results, orient='inde')